In [2]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm
import pandas as pd

In [3]:
original_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity']

new_column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'SeaTemperature', 'RelativeHumidity', 'WaveHeight', 'WavePeriod']

with open('../data/raw_data.csv', 'r') as f1, open('../data/interim_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=original_column_names)
    writer = DictWriter(f2, fieldnames=new_column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in new_column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in new_column_names})


613392it [00:03, 183941.44it/s]


In [16]:
df = pd.read_csv('../data/interim_data.csv')
df.shape

(385474, 9)

In [18]:
# Use the interquartile range to remove the outliers in data.
for column_name in new_column_names:
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]


df.to_csv('../processed_data.csv')
df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,RelativeHumidity,WaveHeight,WavePeriod
count,353442.000000,353442.000000,353442.000000,353442.000000,353442.000000,353442.000000,353442.000000,353442.000000,353442.000000
mean,1015.024020,204.206851,14.360989,20.231081,11.744330,12.521505,82.673260,2.043567,5.911595
std,11.077417,90.471302,6.189358,8.243699,2.880462,2.339600,9.536276,1.117457,1.519769
min,984.534000,0.000000,0.000000,0.000000,2.964000,5.900000,55.000000,0.100000,2.000000
25%,1007.861000,150.000000,10.000000,14.000000,9.697000,10.700000,76.000000,1.200000,5.000000
50%,1015.800000,214.000000,14.000000,20.000000,11.800000,12.500000,83.594000,1.800000,6.000000
75%,1022.800000,270.000000,19.000000,25.000000,14.141000,14.438250,90.000000,2.700000,7.000000
max,1045.400000,360.000000,32.333000,43.000000,20.600000,19.200000,100.000000,5.195000,10.000000
